<a href="https://colab.research.google.com/github/Pratiksha100/AnalyticsProject/blob/main/pyconcorde_upgrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
%pip install git+https://github.com/jvkersch/pyconcorde

In [6]:
# install tweak file
%cp '/content/drive/MyDrive/Analytics Project/Concorde/concorde_class.py' '/content'
%cp '/content/drive/MyDrive/Analytics Project/Concorde/test.py' '/content'
%cp '/content/drive/MyDrive/Analytics Project/Concorde/concorde_to_replace/tsp.py' '/usr/local/lib/python3.10/dist-packages/concorde'
%cp '/content/drive/MyDrive/Analytics Project/Concorde/concorde_to_replace/util.py' '/usr/local/lib/python3.10/dist-packages/concorde'

In [ ]:
'''
%%capture
!cp "/content/drive/MyDrive/Analytics Project/Concorde.zip" "/content"
!unzip Concorde.zip
!bash /content/Concorde/install_concorde.sh
'''

'\n%%capture\n!cp "/content/drive/MyDrive/Analytics Project/Concorde.zip" "/content"\n!unzip Concorde.zip\n!bash /content/Concorde/install_concorde.sh\n'

In [1]:
import numpy as np
import pandas as pd
import math
import os

In [2]:
def create_dist_mat_from_atsp(path_of_file): #input : path of the .atsp file
    # specify the path to the .atsp file
    file_path = path_of_file
    print(file_path)


    # initialize variables
    n_nodes = None
    dist_matrix = []

    # open and read the file
    with open(file_path) as f:
        for line in f:
            # extract the number of nodes
            if "DIMENSION" in line:
                n_nodes = int(line.split(":")[1].strip())
            # extract the distance matrix
            elif "EDGE_WEIGHT_SECTION" in line:
                while True:
                    line = f.readline()
                    if "EOF" in line:
                        break
                    dist_matrix.append(list(map(int, line.split())))
            # continue reading until end of file
            elif "EOF" in line:
                break


    #resizing the distance matrix to a square matrix
    list_of_lists = dist_matrix

    flat_list = []
    for sublist in list_of_lists:
        for item in sublist:
            flat_list.append(item)


    # Example 1D list
    lst = flat_list

    # Calculate the size of the square matrix
    n = len(lst)
    size = int(np.sqrt(n))

    # Reshape the 1D list into a 2D numpy array with the desired shape
    arr = np.reshape(lst, (size, size))

    # Print the resulting array
    dist_mat = np.array(arr)

    print("No of cities : "  , n_nodes)
    print(dist_mat.shape)
    return dist_mat #returns the distance matrix

import pickle

def round_up( lst):
        '''
        input : list of lists having non integral values
        output : list of lists having integral values.
        Function to round up numbers and return as integers
        '''
        rounded_lst = []
        for inner_lst in lst:
            rounded_inner_lst = []
            for num in inner_lst:
                rounded_num = int(round(num))
                rounded_inner_lst.append(rounded_num)
            rounded_lst.append(rounded_inner_lst)
        return rounded_lst

def read_pickle(pickle_path):
    # open the pickle file in read-binary mode
    with open(pickle_path, 'rb') as f:
        pic_obj = pickle.load(f)
    pic_obj = round_up(pic_obj)

    return pic_obj

In [ ]:
filelist = ['dm_Warmsen_Friday_31600-01_scenario_high_all.p',
'dm_Warmsen_Friday_31600-01_scenario_high_letter.p',
'dm_Warmsen_Friday_31600-01_scenario_high_others.p',
'dm_Warmsen_Friday_31600-01_scenario_high_package.p',
'dm_Warmsen_Friday_31600-01_scenario_low_all.p',
'dm_Warmsen_Friday_31600-01_scenario_low_letter.p',
'dm_Warmsen_Friday_31600-01_scenario_low_others.p',
'dm_Warmsen_Friday_31600-01_scenario_low_package.p',
'dm_Warmsen_Friday_31600-01_scenario_medium_all.p',
'dm_Warmsen_Friday_31600-01_scenario_medium_letter.p',]

In [ ]:
distances = read_pickle('/content/drive/MyDrive/Analytics Project/Warsem /Warmsen_data/dm_Warmsen_Friday_31600-01_scenario_high_all.p')
# print(distances)

In [ ]:
%%time
import time

from concorde_class import Concorde

# distances = create_dist_mat_from_atsp('/content/drive/MyDrive/Analytics Project/TSPLIB_ATSP/rbg443.atsp')
# distances = [[0,50,80],[5,0,1],[7,10,0]]
# distances = read_pickle('/content/warmze_31606-12_dienstag_briefe1_distmatrix.p')

path = '/content/drive/MyDrive/Analytics Project/Coding/All_orig_files'

file_dict = dict()
for i in sorted(i for i in os.listdir(path) if i.startswith('dm_Warmsen_31600-03')):
    distances = np.array(read_pickle(os.path.join(path,i)))
    start = time.time()
    concorde = Concorde()

    concorde.run_ProbATSP(distances)

    end = time.time()
    file_dict[i] = {}
    file_dict[i]['obj'] = concorde.optimal_value
    file_dict[i]['time'] = end - start
    file_dict[i]['route'] = concorde.route
    print('-------------------------------------------------------------------------')
    print(f'route: {concorde.route}')
    print(f'value: {concorde.optimal_value} ,file:{i} ,time: {end-start}')

In [ ]:
file_dict

In [1]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres